In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2024-08-07 05:35:53.315222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 05:35:53.315389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 05:35:53.483433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [3]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
submission

,row_id,normal_mild,moderate,severe
0,samples,0.333333,0.333333,0.333333
1,samples,0.333333,0.333333,0.333333
2,samples,0.333333,0.333333,0.333333
3,samples,0.333333,0.333333,0.333333
4,samples,0.333333,0.333333,0.333333
5,samples,0.333333,0.333333,0.333333
6,samples,0.333333,0.333333,0.333333
7,samples,0.333333,0.333333,0.333333
8,samples,0.333333,0.333333,0.333333
9,samples,0.333333,0.333333,0.333333


In [7]:
config = {}
config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'

studies = os.listdir(config['root_file_path'])

In [8]:
studies

['44036939']

In [9]:
test_dict = {}
image_files = []

for dirname, _, filenames in os.walk(config['root_file_path']):
    for filename in filenames:
        test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
        image_files.append(os.path.join(dirname, filename))

In [10]:
test_dict

{'44036939': ['/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/18.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/9.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/22.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/25.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/14.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/11.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/24.dcm',
  '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/23.dcm',
  '/kagg

In [13]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        for file in test_dict[st]:
                            dicom_ds = dcmread
                    #print(st+'_'+s+'_'+c+'_'+v)
                    rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

44036939_spinal_canal_stenosis_l1_l2
44036939_spinal_canal_stenosis_l2_l3
44036939_spinal_canal_stenosis_l3_l4
44036939_spinal_canal_stenosis_l4_l5
44036939_spinal_canal_stenosis_l5_s1
44036939_left_neural_foraminal_narrowing_l1_l2
44036939_right_neural_foraminal_narrowing_l1_l2
44036939_left_neural_foraminal_narrowing_l2_l3
44036939_right_neural_foraminal_narrowing_l2_l3
44036939_left_neural_foraminal_narrowing_l3_l4
44036939_right_neural_foraminal_narrowing_l3_l4
44036939_left_neural_foraminal_narrowing_l4_l5
44036939_right_neural_foraminal_narrowing_l4_l5
44036939_left_neural_foraminal_narrowing_l5_s1
44036939_right_neural_foraminal_narrowing_l5_s1
44036939_left_subarticular_stenosis_l1_l2
44036939_right_subarticular_stenosis_l1_l2
44036939_left_subarticular_stenosis_l2_l3
44036939_right_subarticular_stenosis_l2_l3
44036939_left_subarticular_stenosis_l3_l4
44036939_right_subarticular_stenosis_l3_l4
44036939_left_subarticular_stenosis_l4_l5
44036939_right_subarticular_stenosis_l4_l5


In [10]:
features = []
dicom_ds = dcmread('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm')
img_array = dicom_ds.pixel_array
features.append(np.mean(img_array.T, axis=0))


feature_resized = []
for img in features:
    feature_resized.append(np.resize(img,(128,)))
    
final = np.array(feature_resized)

In [11]:
final.shape

(1, 128)

In [12]:
y_proba = model.predict(final)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step


I0000 00:00:1722921523.439021     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [19]:
y_proba

array([[3.523812e-02, 9.638161e-01, 9.457982e-04]], dtype=float32)

In [3]:
test_dict = {}
image_files = []

for dirname, _, filenames in os.walk('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'):
    for filename in filenames:
        test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
        image_files.append(os.path.join(dirname, filename))

In [4]:
for key, value in test_dict.items():
    print(key)
    features = []
    feature_resized = []
    for file in value:
        print(file)
        dicom_ds = dcmread('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm')
        img_array = dicom_ds.pixel_array
        features.append(np.mean(img_array.T, axis=0))
    
    for img in features:
        feature_resized.append(np.resize(img,(128,)))
        
    final = np.array(feature_resized)
    print(final.shape)
    
    y_proba = model.predict(final)

44036939
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/22.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/25.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/11.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/24.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/23.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classi

In [18]:
np.mean(y_proba, axis=0)

array([3.5238091e-02, 9.6381575e-01, 9.4579888e-04], dtype=float32)